In [1]:
from general_utils import *
from own_utils import *

Notebook general setup done.
Notebook general setup done.
Notebook own setup done.


In [4]:
EbfN0 = 2
im = Image.open('../imgs/input.tif')
imarray = np.array(im)

cimarray, tr = build_huffman_tree(imarray, x)

cimarray_list = [int(bit) for bit in cimarray]
aux = np.matrix(cimarray_list)

decoded_image_bits = []
total_bits = aux.shape[1]
padding_bits = (k - (total_bits % k)) % k

# Pad the aux matrix if necessary
if padding_bits > 0:
    padding = np.zeros((1, padding_bits), dtype=int)
    aux = np.concatenate((aux, padding), axis=1)


# Process aux in chunks of size k
for i in range(0, aux.shape[1], k):
    message_chunk = aux[:, i:i+k] # This should be 1x10
    original_encoded_codeword = encode_message(message_chunk, G) # This should be 1x14

    received_codeword = None
    syndrome = np.matrix(np.ones((1, n - k), dtype=int)) # Initialize with non-zero syndrome to enter loop
    retransmission_count = 0

    # Simulate retransmissions until no error is detected or max retransmissions reached
    while np.any(syndrome) and retransmission_count < max_retransmissions:
        received_codeword = noisy_channel(original_encoded_codeword, n=n, k=k, EbfN0=EbfN0) # This should be 1x14
        syndrome = (received_codeword @ H.transpose()) % 2 # H.transpose() is 14x4, result is 1x4

        if np.any(syndrome):
            retransmission_count += 1 # Error detected, simulate retransmission

    # After successfully receiving a codeword (syndrome is zero) or max retransmissions reached
    if not np.any(syndrome):
        # This assumes the encode_message produces a systematic codeword
        decoded_message_chunk = received_codeword[:, :k] # Extract first k bits (1x10)
        decoded_image_bits.append(decoded_message_chunk)
    else:
        # If max retransmissions reached and error still detected, what to do?
        # Option 1: Append a block of zeros (treat as erasure)
        decoded_image_bits.append(np.zeros((1, k), dtype=int))
        print(f"  Max retransmissions ({max_retransmissions}) reached for chunk starting at bit {i}. Appending zero block.")

        # Option 2: Append the last received (erroneous) message part (might introduce more errors)
        # decoded_message_chunk = received_codeword[:, :k]
        # decoded_image_bits.append(decoded_message_chunk)


In [5]:
# At this point, decoded_image_bits is a list of numpy matrices, each 1xk.
combined_decoded_bits = np.concatenate(decoded_image_bits, axis=1)

# Unpad combined_decoded_bits to get the original number of bits in cimarray
# total_bits = "length of aux before padding"
combined_decoded_bits = combined_decoded_bits[:, :total_bits]
decoded_bits_array = np.array(combined_decoded_bits).flatten()

# Now, perform inverse Huffman decoding to get back the pixel values
reconstructed_pixels = []
current_code_bits = ""

# Use the code_to_pixel_tuple dictionary for decoding
# Iterate through the flattened decoded bits array
for bit in decoded_bits_array:
    current_code_bits += str(bit)
    # Check if the current accumulated bits form a valid Huffman code
    if current_code_bits in tr:
        pixel_tuple = tr[current_code_bits] # Get the original pixel tuple for the found code
        reconstructed_pixels.extend(pixel_tuple) # Add the pixel values from the tuple to our list
        current_code_bits = "" # Reset the accumulated bits for the next code

# Convert the list of reconstructed pixels to a numpy array
reconstructed_array = np.array(reconstructed_pixels, dtype=np.uint8)

# Check if the number of reconstructed pixels matches the original image size
# If not, pad or truncate the reconstructed array to match the original size (size mismatch indicates errors)
if reconstructed_array.size < imarray.size:
    padding_size = imarray.size - reconstructed_array.size
    reconstructed_array = np.pad(reconstructed_array, (0, padding_size), 'constant', constant_values=0)
    print(f"Padded reconstructed pixels with {padding_size} zeros to match original image size.")
elif reconstructed_array.size > imarray.size:
     reconstructed_array = reconstructed_array[:imarray.size]
     print(f"Truncated reconstructed pixels by {reconstructed_array.size - imarray.size} to match original image size.")

decoded_image = reconstructed_array.reshape(imarray.shape)
decoded_image[decoded_image == 1] = 255 # map 1 to 255 in RGB

decoded_image_pil = Image.fromarray(decoded_image)
decoded_image_pil.save('../imgs/decoded.tif')
print("Image decoded and saved as 'decoded.tif'")

Padded reconstructed pixels with 52 zeros to match original image size.
Image decoded and saved as 'decoded.tif'
